In [5]:
import os
import glob
import pandas as pd
import numpy as np
import datetime

# 절대모멘텀 코드 재활용

In [10]:
def data_preprocessing(sample, ticker, base_date):   
    sample['CODE'] = ticker # 종목코드 추가
    sample = sample[sample['Date'] >= base_date][['Date','CODE','Adj Close']].copy() # 기준일자 이후 데이터 사용
    sample.reset_index(inplace= True, drop= True)
    # 기준년월 
    sample['STD_YM'] = sample['Date'].map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m')) 
    sample['1M_RET'] = 0.0 # 수익률 컬럼
    ym_keys = list(sample['STD_YM'].unique()) # 중복 제거한 기준년월 리스트
    return sample, ym_keys

In [11]:
def create_trade_book(sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes].copy()
    book['STD_YM'] = book.index.map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))
    for c in sample_codes:
        book['p '+c] = ''
        book['r '+c] = ''
    return book

In [13]:

# 상대모멘텀 tradings
def tradings(book, s_codes):
    std_ym = ''
    buy_phase = False
    # 종목코드별 순회
    for s in s_codes : 
        print(s)
        # 종목코드 인덱스 순회
        for i in book.index:
            # 해당 종목코드 포지션을 잡아준다. 
            if book.loc[i,'p '+s] == '' and book.shift(1).loc[i,'p '+s] == 'ready ' + s:
                std_ym = book.loc[i,'STD_YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있으면 매수상태를 유지한다.
            if book.loc[i,'p '+s] == '' and book.loc[i,'STD_YM'] == std_ym and buy_phase == True : 
                book.loc[i,'p '+s] = 'buy ' + s
            
            if book.loc[i,'p '+ s] == '' :
                std_ym = None
                buy_phase = False
    return book

In [14]:
def multi_returns(book, s_codes):
    # 손익 계산
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}
    
    for i in book.index:
        for s in s_codes:
            if book.loc[i, 'p ' + s] == 'buy '+ s and \
            book.shift(1).loc[i, 'p '+s] == 'ready '+s and \
            book.shift(2).loc[i, 'p '+s] == '' :     # long 진입
                buy_dict[s] = book.loc[i, s]
#                 print('진입일 : ',i, '종목코드 : ',s ,' long 진입가격 : ', buy_dict[s])
            elif book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i, 'p '+s] == 'buy '+ s:     # long 청산
                sell_dict[s] = book.loc[i, s]
                # 손익 계산
                rtn = (sell_dict[s] / buy_dict[s]) -1
                book.loc[i, 'r '+s] = rtn
                print('개별 청산일 : ',i,' 종목코드 : ', s , 'long 진입가격 : ', buy_dict[s], ' |  long 청산가격 : ',\
                      sell_dict[s],' | return:', round(rtn * 100, 2),'%') # 수익률 계산.
            if book.loc[i, 'p '+ s] == '':     # zero position || long 청산.
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0


    acc_rtn = 1.0        
    for i in book.index:
        rtn  = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s: 
                # 청산 수익률계산.
                count += 1
                rtn += book.loc[i, 'r '+s]
        if (rtn != 0.0) & (count != 0) :
            acc_rtn *= (rtn /count )  + 1
            print('누적 청산일 : ',i,'청산 종목수 : ',count, \
                  '청산 수익률 : ',round((rtn /count),4),'누적 수익률 : ' ,round(acc_rtn, 4)) # 수익률 계산.
        book.loc[i,'acc_rtn'] = acc_rtn
    print ('누적 수익률 :', round(acc_rtn, 4))

# 상대모멘텀


In [7]:
files = glob.glob('./us_etf_data/*.csv')
print(files)

['./us_etf_data/BND.csv', './us_etf_data/AAPL.csv', './us_etf_data/AMZN.csv']


필요한 데이터프레임 생성

Montly 데이터 저장

In [17]:
month_last_df = pd.DataFrame(columns=['Date', 'Code', '1M_RET'])
stock_df = pd.DataFrame(columns=['Date', 'CODE','Adj Close'])

In [18]:
for file in files:
    """
    데이터 저장 경로에 있는 개별 종목들을 읽어온다.
    """
    if os.path.isdir(file):
        print('%s <DIR> '%file)
    else:
        folder, name = os.path.split(file)
        head, tail = os.path.splitext(name)
        print(file)
        read_df = pd.read_csv(file) # 경로를 읽은 데이터를 하나씩 읽어들인다.
        
        # 1단계. 데이터 가공
        price_df, ym_keys = data_preprocessing(read_df,head,base_date='2010-01-02')
        # 가공한 데이터 붙이기.
        stock_df = stock_df.append(price_df.loc[:,['Date','CODE','Adj Close']],sort=False)
        # 월별 상대모멘텀 계산을 위한 1개월간 수익률 계산
        for ym in ym_keys:
            m_ret = price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],'Adj Close'] \
            / price_df.loc[price_df[price_df['STD_YM'] == ym].index[0],'Adj Close'] 
            price_df.loc[price_df['STD_YM'] == ym, ['1M_RET']] = m_ret
            month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1],\
                                                              ['Date','CODE','1M_RET']])


./us_etf_data/BND.csv
./us_etf_data/AAPL.csv
./us_etf_data/AMZN.csv


In [24]:
month_ret_df = month_last_df.pivot('Date','CODE','1M_RET').copy()
month_ret_df = month_ret_df.rank(axis=1, ascending=False, method="max", pct=True)

In [25]:
month_ret_df.head()

CODE,AAPL,AMZN,BND
Date,,,
2010-01-25,1.000000,0.666667,0.333333
2010-02-22,0.333333,0.666667,1.000000
2010-03-29,0.333333,0.666667,1.000000
2010-04-26,0.333333,1.000000,0.666667
2010-05-31,0.333333,1.000000,0.666667


상위 40%에 드는 종목들만 신호 목록

In [26]:
month_ret_df = month_ret_df.where(month_ret_df<0.4, np.nan)
month_ret_df.fillna(0, inplace=True)
month_ret_df[month_ret_df!=0] = 1 #?????
stock_codes = list(stock_df['CODE'].unique())

In [28]:
month_ret_df.head()

CODE,AAPL,AMZN,BND
Date,,,
2010-01-25,0.0,0.0,1.0
2010-02-22,1.0,0.0,0.0
2010-03-29,1.0,0.0,0.0
2010-04-26,1.0,0.0,0.0
2010-05-31,1.0,0.0,0.0


In [29]:
#3단계. 신호 목록으로 트레이딩 + 포지셔닝

sig_dict = dict()
for date in month_ret_df.index:
  ticker_list = list(month_ret_df.loc[date, month_ret_df.loc[date, :] >= 1.0].index)
  sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date','CODE','Adj Close').copy()
book = create_trade_book(stock_c_matrix, list(stock_df['CODE'].unique()))

In [30]:
for date, values in sig_dict.items():
  for stock in values:
    book.loc[date, 'p '+stock] = 'ready ' + stock

In [31]:
book.head()

CODE,BND,AAPL,AMZN,STD_YM,p BND,r BND,p AAPL,r AAPL,p AMZN,r AMZN
Date,,,,,,,,,,
2010-01-04,57.169640,6.510078,133.520004,2010-01,,,,,,
2010-01-11,57.452190,6.324279,127.139999,2010-01,,,,,,
2010-01-18,57.604393,6.073064,121.430000,2010-01,,,,,,
2010-01-25,57.676796,5.898319,125.410004,2010-01,ready BND,,,,,
2010-02-01,57.662350,6.002738,117.389999,2010-02,,,,,,


In [32]:
book = tradings(book, stock_codes)
multi_returns(book, stock_codes)

BND
AAPL
AMZN
개별 청산일 :  2010-03-01  종목코드 :  BND long 진입가격 :  57.66235  |  long 청산가격 :  57.762623  | return: 0.17 %
개별 청산일 :  2010-07-05  종목코드 :  AAPL long 진입가격 :  6.724133  |  long 청산가격 :  7.973139999999999  | return: 18.57 %
개별 청산일 :  2010-09-06  종목코드 :  BND long 진입가격 :  59.823119999999996  |  long 청산가격 :  60.683932999999996  | return: 1.44 %
개별 청산일 :  2011-02-07  종목코드 :  AMZN long 진입가격 :  142.440002  |  long 청산가격 :  189.25  | return: 32.86 %
개별 청산일 :  2011-04-04  종목코드 :  AAPL long 진입가격 :  10.959155  |  long 청산가격 :  10.289966999999999  | return: -6.11 %
개별 청산일 :  2011-06-06  종목코드 :  AMZN long 진입가격 :  184.71000700000002  |  long 청산가격 :  186.529999  | return: 0.99 %
개별 청산일 :  2011-07-04  종목코드 :  BND long 진입가격 :  62.476443999999994  |  long 청산가격 :  62.460190000000004  | return: -0.03 %
개별 청산일 :  2011-08-01  종목코드 :  AMZN long 진입가격 :  218.27999900000003  |  long 청산가격 :  202.699997  | return: -7.14 %
개별 청산일 :  2011-09-05  종목코드 :  AAPL long 진입가격 :  11.474173  |  long 청산가격 :  11.59272  | retu